# 1 Files management
## 1.1 recursive file management
If you want to manage file in one directory recursively, you can use `mapfile` command.

Remove files recursively:
```
mapfile -t rmlist < rmlist.txt
size=${#rmlist[@]}
for (( i = 0; i < size; i++ ));
do
    rm -r ${rmlist[i]}
done
```
Dowload files recursively:
```
mapfile -t dllist < dllist.txt
mapfile -t dirlist < dirlist.txt
size=${#dllist[@]}
for (( i = 0; i < size; i++ ));
do
    wget ftp://download.big.ac.cn/gsa2/CRA012590/2/${dllist[i]} -P $1${dirlist[i]}
done
```
There is a script I wrote to remove `.bam` files in a directory recursively:

In [ ]:
%%bash
#! /bin/bash

# File name: rm.bam.sh
# Remove the primary .bam files in a directory recursively.
## $1: number
## $2: /path/to/index
## $3: /path/to/fastq_data

mapfile -t tag < $1
size=${#tag[@]}
echo "Tags loaded: ${#tag[@]}"
for ((i=0; i<size; i++));
  do
  echo "Processing tag: ${tag[i]}"
  if [ -f ${tag[i]}.rmdup.bam ]; then
    echo "Removing file: ${tag[i]}.bam"
    rm -r ${tag[i]}.bam
  fi
done


Also, there is a script I wrote to copy files in a directory recursively:

In [ ]:
%%bash
#! /bin/bash

# File name: cp-for
# This script is used to copy the fastq files recursively.
## $1: /file/location
## $2: /path/to/destination
## $3: files

files=$3
size=${#files[@]}
for (( i = 0; i < size; i++ )); do
  cp -ax "$1/${files[i]}" "$2"
done

## 1.2 file comparison
To check is two directories contain same files, you can first save the file name into different files:
- `ls 115/ > name.115`
Then, you can use `grep -F -x -f name.115 name.203` to check if the files in `name.115` are also in `name.203`.
- the directions are the same.
To check the difference of two files, you can use `diff` command:
- `diff -q file1 file2` to check if the files are different.

## 1.3 file package and compression
Use `tar` to pack files: `tar -cvf {file}.tar {dir}`
Use `tar` to unpack files: `tar -xvf {file}.tar`

Use `gzip` to compress files: `gzip {file}`
Use `pigz` to compress files: `pigz -3 {file}`

Use `tar` to unpack compressed files: `tar -xzvf {file}.tar.gz`


# 2 Vim
阅读二进制文件：`vim -b {file}`
- 将内容转化为16进制：`:%!xxd`
- 将内容转化回文本格式：`:%!xxd -r`


# 3 conda
查看当前conda有哪些环境：`conda env list`


# 4 git
## 4.1 git setting
To set the git proxy, you can use the following command:
- `git config --global http.proxy http://10.207.199.218:7890`

To unset the git proxy, you can use the following command:
- `git config --global --unset http.proxy`
- 

# 5 Coding Environment

## 5.1 Java

You can set different versions of Java for one user and transfer between by simple commands. To do this, first, you need to install openJDKs from various sources. Here I list some:
1. install openJDK from oracle: https://www.oracle.com/java/technologies/downloads/
    - remember to install the x64 version
2. install from localhost:
    - `scp D:\Zheng\Documents\2_NBS\pack\* *@*:~/lib`

After you install the compressed archive openJDK to server, you can do the following steps:
1. `cd ~/lib`
2. `tar -xzvf jdk-8u451-linux-x64.tar.gz`
3. `tar -xzvf jdk-17.0.15_linux-x64_bin.tar.gz`
4. `tar -xzvf jdk-21_linux-x64_bin.tar.gz`
5. `mkdir jvm pack`
6. `mv jdk-17.0.15_linux-x64_bin.tar.gz jdk-21_linux-x64_bin.tar.gz jdk-8u451-linux-x64.tar.gz pack/`
7. `mv mv jdk1.8.0_451/ jvm/jdk-8`
8. `mv jdk-17.0.15/ jvm/jdk-17`
9. `mv jdk-21.0.7/ jvm/jdk-21`

Then, you can set `~/.bashrc` with the following lines:
```
# java
# different java version setting
## 1. fields
export JAVA_8_HOME=~/lib/jvm/jdk-8
export JAVA_17_HOME=~/lib/jvm/jdk-17
export JAVA_21_HOME=~/lib/jvm/jdk-21
## 2. default setting
export JAVA_HOME=$JAVA_8_HOME
export PATH=$JAVA_HOME/bin:$PATH
## 3. set version change commands
alias java8='export JAVA_HOME=$JAVA_8_HOME && export PATH=$JAVA_HOME/bin:$PATH'
alias java17='export JAVA_HOME=$JAVA_17_HOME && export PATH=$JAVA_HOME/bin:$PATH'
alias java21='export JAVA_HOME=$JAVA_21_HOME && export PATH=$JAVA_HOME/bin:$PATH'
```
Finally, run `source ~/.bashrc` to load the new settings.

You can use `java8`, `java17` and `java21` to switch between different versions of Java and check the version of Java by using `java -version`, `echo $JAVA_HOME`, and `which java` command.


# 6 perf

## 6.1 install perf

To install `perf`, you should have root permission.

First, I tried `sudo yum install -y perf` to install, but it failed with the following error:
```
已加载插件：fastestmirror, langpacks
Loading mirror speeds from cached hostfile
Could not retrieve mirrorlist http://mirrorlist.centos.org/?release=7&arch=x86_64&repo=os&infra=stock error was
14: curl#6 - "Could not resolve host: mirrorlist.centos.org; 未知的错误"


 One of the configured repositories failed (未知),
 and yum doesn't have enough cached data to continue. At this point the only
 safe thing yum can do is fail. There are a few ways to work "fix" this:

     1. Contact the upstream for the repository and get them to fix the problem.

     2. Reconfigure the baseurl/etc. for the repository, to point to a working
        upstream. This is most often useful if you are using a newer
        distribution release than is supported by the repository (and the
        packages for the previous distribution release still work).

     3. Run the command with the repository temporarily disabled
            yum --disablerepo=<repoid> ...

     4. Disable the repository permanently, so yum won't use it by default. Yum
        will then just ignore the repository until you permanently enable it
        again or use --enablerepo for temporary usage:

            yum-config-manager --disable <repoid>
        or
            subscription-manager repos --disable=<repoid>

     5. Configure the failing repository to be skipped, if it is unavailable.
        Note that yum will try to contact the repo. when it runs most commands,
        so will have to try and fail each time (and thus. yum will be be much
        slower). If it is a very temporary problem though, this is often a nice
        compromise:

            yum-config-manager --save --setopt=<repoid>.skip_if_unavailable=true

Cannot find a valid baseurl for repo: base/7/x86_64
```
Then I tried to install `kernel-tools` and `kernel-debuginfo`. 
1. First, I tried `uname -a` to check the centOS version of 115 server, which is `3.10.0-1127.el7.x86_64`. 
2. Then I search https://mirrors.aliyun.com/centos/7/os/x86_64/Packages/ for two packages, but I can only find the `kernel-tools-3.10.0-1160.el7.x86_64.rpm` version. 
3. With more efforts, I found that all the `rpm` files are in https://vault.centos.org/7.7.1908/os/x86_64/Packages/.
4. Finally, I found
    - https://vault.centos.org/7.7.1908/os/x86_64/Packages/kernel-tools-3.10.0-1062.el7.x86_64.rpm
    - https://vault.centos.org/7.7.1908/os/x86_64/Packages/kernel-debug-3.10.0-1062.el7.x86_64.rpm




